# Assignment 4 - CS535 - Natural Language Processing
# Name: Muhammad Ahsan Farooqui
# Roll Number : 20I-2207



# Running Instructions: 
* Simply Running all cells should run the whole script. 
* First Part of the assignment contains all functions that are required to run. 
* In second part, all functions are called
* In third part, a few cases are tested. 


# Challenges Faced: 

* The corpus size was huge and it forced me to explore new libraries
* The corpus data.txt itself contains many slang words and non-standard spellings. 

# Part 1



---



In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

## Adding Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import os
!pip install iteration-utilities
from iteration_utilities import deepflatten
import pickle
import iteration_utilities
from collections import Counter
from itertools import islice
import time
from collections import OrderedDict
import operator
import re


## Importing Data

In [2]:
def get_data():
  with open('data.txt', 'r') as f:
      passage = f.readlines()

  overall_data = passage

  with open('misspellings.txt', 'r') as f:
      reader = csv.reader(f)
      misspellings = list(reader)
  f.close()
  overall_data2 = ["# "+x+" $" for x in overall_data]
  words_corpus = [re.split("\s",lst) for lst in overall_data2]
  #words_corpus = [x.split() for x in overall_data2]
  words_corpus = list(deepflatten(words_corpus,depth=1))
  return words_corpus,misspellings

## Calculating Word Unigrams

In [3]:
def get_word_unigrams(words_corpus):
  word_unigrams = Counter(words_corpus)
  word_unigrams = dict(word_unigrams)
  word_unigram_probability = dict()
  word_unigram_wordlength = dict()
  r = sum(list(word_unigrams.values()))
  for i in word_unigrams.keys():
    word_unigram_probability[i] = word_unigrams[i]/r
    word_unigram_wordlength[i] = len(i)
  return word_unigrams,word_unigram_probability,word_unigram_wordlength

## Calculating Character Unigrams and Bigrams

In [4]:
def get_character_unigrams(words_corpus):
  characters_corpus = list(deepflatten(words_corpus,depth=3))
  from collections import Counter
  character_counters = Counter(characters_corpus)
  character_dict = dict(character_counters)
  char_unigrams_dict = dict()
  return characters_corpus,character_dict

In [5]:
def get_character_bigrams(characters_corpus):
  bigram_counts = Counter(zip(characters_corpus, islice(characters_corpus, 1, None)))
  bigram_counts = dict(bigram_counts)
  return bigram_counts

## Now Creating Confusion Matrix

### Some functions to help create dictionaries. 

In [6]:
def initialize(dict1,id=True):
    if(id):
      for i in range(97,123):
          dict1[chr(i)] = dict()
          for k in range(97,123):
              dict1[chr(i)][chr(k)] = 0
    else:
      a = list(range(97,123))
      a.append(35)
      a.append(36)
      for i in a:
          dict1[chr(i)] = dict()
          #print(chr(i))
          for k in a:
              dict1[chr(i)][chr(k)] = 0

    return dict1

def get_dictionaries():
    insert = {}
    substitute = {}
    delete = {}
    transpose = {}
    insert = initialize(insert,id=False)
    substitute = initialize(substitute,id=False)
    delete = initialize(delete,id=False)
    transpose = initialize(transpose,id=False)
    return insert,substitute,delete,transpose

### Defining the Transpose, Delete, Substitute and Transpose Functions

In [7]:
def transpose1(correct,incorrect):
    b = incorrect
    if(len(correct)==len(incorrect)):
        for i,j in enumerate(zip(incorrect,correct)):
            if(j[0]==j[1]):
                continue
            else:
                a = correct
                a = list(a)
                tmp = a[i]
                if(i<len(correct)-1):
                  a[i] = a[i+1]
                  a[i+1]=tmp
                b = incorrect
                a = "".join(a)
                if(a==b):
                  return [("transpose",j[0],j[1])]
        return []
    else:
        return [] 

def delete1(correct,incorrect):
    delete_dict = []
    correct = "#"+correct+"$"
    incorrect = "#"+incorrect+"$"
    if(len(correct)>len(incorrect)):
        for i in range(len(correct)):
            if(correct[0:i]+correct[i+1:]==incorrect):
                delete_dict.append(("delete",correct[i-1],correct[i]))
        return delete_dict
    else:
        return delete_dict
def substitute1(correct,incorrect):
    if(len(correct)==len(incorrect)):
        b = incorrect
        for i,j in enumerate(zip(correct,incorrect)):
            if(j[0]==j[1]):
                continue
            else:
              a = list(correct)
              
              a[i] = j[1]
              a = "".join(a)
              if(a==b):
                return [("substitute",j[1],j[0])]
        return []
    else:
        return []
def insert1(correct,incorrect):
    insert_list = []
    correct = "#"+correct+"$"
    incorrect = "#"+incorrect+"$"
    if(len(incorrect)>len(correct)):
        for i in range(len(incorrect)):
            if(incorrect[0:i]+incorrect[i+1:]==correct):
                insert_list.append(("insertion",correct[i-1],incorrect[i]))
        return insert_list
    else:
        return insert_list

In [22]:
get_operations("hello","hellp")

[('substitute', 'p', 'o')]

### Returning operations i.e. the insert, correct, delete and substitute against two strings

In [8]:
def get_operations(correct,incorrect):
    if transpose1(correct,incorrect)!=[]:
        return transpose1(correct,incorrect)
    elif delete1(correct,incorrect)!=[]:
        return (delete1(correct,incorrect))
    elif substitute1(correct,incorrect)!=[]:
        return (substitute1(correct,incorrect)) 
    elif insert1(correct,incorrect)!=[]:
        return (insert1(correct,incorrect))
    else:
        return [("none",0,0)]

### Function that takes into account the misspellings and calculates the error functions. 

* Note: it is trained on the given misspellings file that has first line equal to [Correct, Incorrect] which is omitted

In [9]:
def generate_matrix(misspellings):
  insert,substitute,delete,transpose = get_dictionaries()
  for i in misspellings[1:]:
      word = i[0]
      others=i[1].split()
      others = [a.strip() for a in others]
      if(len(others)==1):
        others = list(others)
      for j in others:
          get_opss = get_operations(word,j)
          for get_ops in get_opss: 
            if(get_ops[0]=="insertion"):
              insert[get_ops[1]][get_ops[2]] = insert[get_ops[1]][get_ops[2]] + 1
            elif(get_ops[0]=="delete"):
              delete[get_ops[1]][get_ops[2]] = delete[get_ops[1]][get_ops[2]] + 1
            elif(get_ops[0]=="substitute"):
              substitute[get_ops[1]][get_ops[2]] = substitute[get_ops[1]][get_ops[2]] + 1
            elif(get_ops[0]=="transpose"):
              transpose[get_ops[1]][get_ops[2]] = transpose[get_ops[1]][get_ops[2]] + 1
            
  return insert,substitute,delete,transpose

### Once we get the matrix values, dividing it by the character unigrams or bigrams as mentioned in paper.

In [10]:
def calculate_matrix_probabilities(insert,substitute,delete,transpose,char_unigrams_dict,character_bigram_counts):
  for key in insert.keys():
    for key2 in insert[key]:
      if(insert[key][key2])>0:
        insert[key][key2] = insert[key][key2]/char_unigrams_dict[key]
  for key in substitute:
    for key2 in substitute[key]:
      if(substitute[key][key2]>0):
        substitute[key][key2] =substitute[key][key2]/char_unigrams_dict[key2]
  for key in delete:
    for key2 in delete[key]:
      if(delete[key][key2]>0):
        delete[key][key2] = delete[key][key2]/character_bigram_counts[(key,key2)]
  for key in transpose:
    for key2 in transpose[key]:
      if(transpose[key][key2]>0):
        transpose[key][key2] = transpose[key][key2]/character_bigram_counts[(key,key2)]
  return insert,substitute,delete,transpose

## Get Optimum words. 
* this function gets words unigrams, the word iteself and finds out the best word. 

In [11]:
def get_optimum_words(word,word_unigrams,word_unigram_wordlength):
  shortlisted_words = { your_key: word_unigram_wordlength[your_key] for your_key in word_unigram_wordlength.keys() 
  if((len(your_key) in [len(word)-2,len(word)-1,len(word),len(word)+1,len(word)+2])
    and (get_operations(your_key,word)[0][0]!='none')) }
  ops =[]
  if(word in word_unigram_wordlength.keys()):
    return word,ops
  for i in shortlisted_words.keys():
    ops.append(get_operations(i,word))
  if(len(shortlisted_words)==0):
    return word,ops
  return shortlisted_words,ops


## The below function takes all candidate words and calculates the probabilities

In [12]:
def get_final_word(optimum_words,your_word,insert,substitute,delete,transpose,word_unigram_probability):
  score = dict()
  for i in optimum_words.keys():
    ops = get_operations(i,your_word)
    a = 0
    for j in ops:
      if(j[0]=="insertion"):
        if(i in score.keys()):
          thiscalculate = insert[j[1]][j[2]] * word_unigram_probability[i]
          if(thiscalculate>score[i]):
            score[i]=thiscalculate
        else:
          score[i] = insert[j[1]][j[2]] * word_unigram_probability[i]

      elif(j[0]=="substitute"):
        if(i in score.keys()):
          thiscalculate = substitute[j[1]][j[2]] * word_unigram_probability[i]
          if(thiscalculate>score[i]):
            score[i]=thiscalculate
        else:
          score[i] = substitute[j[1]][j[2]] * word_unigram_probability[i]

      elif(j[0]=="delete"):
        if(i in score.keys()):
          thiscalculate = delete[j[1]][j[2]] * word_unigram_probability[i]
          if(thiscalculate>score[i]):
            score[i]=thiscalculate
        else:
          score[i] = delete[j[1]][j[2]] * word_unigram_probability[i]

      elif(j[0]=="transpose"):
        if(i in score.keys()):
          thiscalculate = transpose[j[1]][j[2]] * word_unigram_probability[i]
          if(thiscalculate>score[i]):
            score[i]=thiscalculate
        else:
          score[i] = transpose[j[1]][j[2]] * word_unigram_probability[i]
  return score


## This is the Final Function that gets a word and returns an optimum word

In [13]:
def spell_checker(word):
  word = word.lower()
  try:
    shortlisted_words,ops = get_optimum_words(word,word_unigrams,word_unigram_wordlength)
  except:
    return word
  try:
    a = shortlisted_words.keys()
  except:
    return word
  try:
    del shortlisted_words["#"]
    del shortlisted_words["$"]
    del shortlisted_words[""]
  except:
    pass
  spell_check=get_final_word(shortlisted_words,word,insert,substitute,delete,transpose,word_unigram_probability)
  if(len(spell_check)==0):
    return word
  #final_dict = OrderedDict(sorted(spell_check.items(), key=operator.itemgetter(1),reverse=True))
  return max(spell_check.items(), key=operator.itemgetter(1))[0]



# Part 2


---


## Putting it all together!

* First line is getting data and also splitting sentences to words
* Second line is getting word level unigrams
* Third line is getting character unigrams
* Fourth is getting character bigrams
* Fifth is getting the error matrices 
* Sixth is calculating the matrix probabilities

In [14]:
words_corpus,misspellings = get_data()
word_unigrams,word_unigram_probability,word_unigram_wordlength = get_word_unigrams(words_corpus)
characters_corpus,char_unigrams_dict = get_character_unigrams(words_corpus)
character_bigram_counts = get_character_bigrams(characters_corpus)
insert,substitute,delete,transpose = generate_matrix(misspellings)
insert_p,substitute_p,delete_p,transpose_p = calculate_matrix_probabilities(insert,substitute,delete,transpose,char_unigrams_dict,character_bigram_counts)


# Part 3

---

## Now Testing on some words

In [15]:
def testing(sentence):
  words = sentence.split()
  try:
    words= [spell_checker(i) for i in words]
    correct_sentence = " ".join(words)
    return correct_sentence
  except:
    return sentence

In [16]:
sentences = ["hmune makret jnaa taha",
             "whd ghr k pass ha",
             "saareye pakistaani shareeef haainn",
             "Poora khadnaaan uthaa kr aa gayay haiiin ",
             "Utla chro kotwal ko datne",
             "hum lahore ki sarko par awara grdi krre hain",
             "Islamabd me das dafatr hain",
             "i 10 mulk hain",
             "Mian Sb london chalayye gayeye ",
             "Adaalat nep ghyeb krdi ha ",
             "h kahsmir gye",
             "macher pankhay se takraa gya",
             'asdfg qwert asdf zxcvb qwert 1234', #case where no alternate is available for any word
             'idnia ke sba say bray netwrok airtel ka hissaa bnen'] 
for i in sentences:
  print(testing(i))


humne market jana tha
who ghr ke pass ha
saarey pakistani shareef hainn
poora khadnaaan uthaa kr aa gaya haiii
ulta chor kotwal ko date
hum lahore ki sarko par awara gardi krre hain
islamabad me das daftar hain
ki 10 mulk hain
mian sb london chalaye gayee
adalat ne gayeb krdi ha
ho kashmir gye
machar pankha se takra gya
asdfg qwert asd zxcvb qwert 1234
india ke sab say baray network airtel ka hissa baen
